In [1]:
#reference: https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/

import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# local variables
sample_data_path = r'C:\Users\steve\Documents\256\Data\sample_data/'

# user data
user = pd.read_csv(sample_data_path + 'users_sample.csv')
# jobs data
jobs = pd.read_csv(sample_data_path + 'jobs_sample_cleared.csv')
# user history data 
user_history = pd.read_csv(sample_data_path + 'user_history_sample.csv')
# apply history data 
app = pd.read_csv(sample_data_path + 'apps_sample.csv')

## Clean up NA value

In [2]:
jobs.isnull().sum()

Unnamed: 0        0
JobID             0
Title             0
Description       1
Requirements    201
State             0
City              0
dtype: int64

In [3]:
jobs.Description = jobs.Description.fillna("")

In [4]:
jobs.isnull().sum()

Unnamed: 0        0
JobID             0
Title             0
Description       0
Requirements    201
State             0
City              0
dtype: int64

### Build TFT Doc vector

In [5]:
#Combine the column u need (adding city and state increase the word size to 180,000 )
documents = []
for i in range (len(jobs)):
    text = "{} {} {} {}".format(jobs.loc[i, 'Title'],
                                   jobs.loc[i, 'Description'],
                                   jobs.loc[i, 'State'],
                                   jobs.loc[i, 'City'])
    documents.append(text)
documents

['Immediate Opening Immediate Opening - AR Clerk is needed  foward resumes to hr.wallsfurniture@aol.com hr.wallsfurniture@aol.com WEB OS69883Accounts Receivable/Cash Applications Clerk is needed,candidates Must have advanced knowledge of computers and peripheral office devices.For immediate consideration candidates should foward resumes to : hr.wallsfurniture@aol.com                                                  Source - Orlando Sentinel FL Orlando',
 'Administrative Assistant ADMINISTRATIVE ASSISTANT Part Time The West Orange Healthcare District has a part time Administrative Assistant position to support the Executive Director and the Board of Trustees by performing reception, secretarial, administrative and financial functions as needed. Must have 3+ years of related experience and be proficient in Microsoft Word, Excel and Power Point. Some working knowledge of QuickBooks is a plus. Send resume to: Kenneth.harker@healthcentral.org EOE Source - Orlando Sentinel FL Orlando',
 "Dom

## TFIDF Initialize

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
# settings that you use for count vectorizer will go here
tfidf_vectorizer = TfidfVectorizer(use_idf=True, stop_words='english')
 
# just send in all your docs here
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(documents)
tfidf_vectorizer_vectors

<3915x24563 sparse matrix of type '<class 'numpy.float64'>'
	with 377066 stored elements in Compressed Sparse Row format>

In [7]:
# place tf-idf values in a pandas data frame
tfdf_2 = pd.DataFrame(tfidf_vectorizer_vectors[0].T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
sorted_tfdf_2 = tfdf_2.sort_values(by=["tfidf"],ascending=False)
valid_tfdf_2 = sorted_tfdf_2[sorted_tfdf_2['tfidf'] > 0]
valid_tfdf_2.to_csv(sample_data_path + 'tfidf2_jd.csv')
#valid_tfdf_2

## get All valid TFIDF Word from the entire document

In [8]:
# initialize empty set
tmp_list = [];
All_KeyWords = set(tmp_list)

#initialize a list of each job id's dictionary
job_vector_list = []

#for test purpose define finite job example
total_jobs = len(jobs)

for i in range(total_jobs):
    print(i)
    vector = tfidf_vectorizer_vectors[i]
    tfdf_2 = pd.DataFrame(vector.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
    sorted_tfdf_2 = tfdf_2.sort_values(by=["tfidf"],ascending=False)
    valid_tfdf_2 = sorted_tfdf_2[sorted_tfdf_2['tfidf'] > 0]
    
    #convert pd to dictionary and store in list
    dictionary = valid_tfdf_2.T.to_dict('list')
    job_vector_list.append(dictionary)
    
    #convert pd to set
    indexNamesArr = valid_tfdf_2.index.values
    All_KeyWords = All_KeyWords | set(indexNamesArr)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063


3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708


In [9]:
print(len(All_KeyWords))
print(len(job_vector_list))
# print(All_KeyWords)

24563
3915


## First Similarity : Scalar Product
### x1 = first recruiter, x2 = second recruiter

In [27]:
def sim(x1, x2):
    list2=[] #debug purpose
    total_sum=0
    for key in job_vector_list[x1]:
        if(key in job_vector_list[x2]):
            first_tfidf = job_vector_list[x1][key][0]
            second_tfidf = job_vector_list[x2][key][0]
            list2.append(key) #debug purpose
            total_sum = total_sum + first_tfidf * second_tfidf
    return total_sum
sim(0, 1)

0.1019244238346225

## Drop extra column and constrain job

In [13]:
#Drop extra column and constrain job
sample_app_undrop = app[app['JobID']<=  jobs.iloc[total_jobs-1][1] ]
sample_app = sample_app_undrop.drop(['Split','Unnamed: 0', 'WindowID', 'ApplicationDate'], axis=1)
print(len(sample_app))
print(len(sample_app.JobID.unique()))

4286
3916


### Split Train and Test Data set

In [14]:
sample_app_copy = sample_app.copy()
train_sample_app = sample_app_copy.sample(frac=0.8, random_state=0)
test_sample_app = sample_app_copy.drop(train_sample_app.index)

### Build Dictionary that map jobID to jobIndex (both)

In [15]:
JobID_to_Index = {}
Index_to_JobID = {}
for index in range(len(jobs)):
#     if(jobs.iloc[index][1] == 562413):
#         print("find")
#     print(index, " ", jobs.iloc[index][1])
    JobID_to_Index[jobs.iloc[index][1]] = index
    Index_to_JobID[index] = jobs.iloc[index][1]

In [16]:
user

,Unnamed: 0,UserID,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,0,47,1,Train,Paramount,CA,US,90723,High School,NaN,1999-06-01 00:00:00,3,10.0,Yes,No,0
1,1,72,1,Train,La Mesa,CA,US,91941,Master's,Anthropology,2011-01-01 00:00:00,10,8.0,Yes,No,0
2,2,80,1,Train,Williamstown,NJ,US,08094,High School,Not Applicable,1985-06-01 00:00:00,5,11.0,Yes,Yes,5
3,3,98,1,Train,Astoria,NY,US,11105,Master's,Journalism,2007-05-01 00:00:00,3,3.0,Yes,No,0
4,4,123,1,Train,Baton Rouge,LA,US,70808,Bachelor's,Agricultural Business,2011-05-01 00:00:00,1,9.0,Yes,No,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,23125,1,Train,Eldersburg,MD,US,21784,Bachelor's,Sociology,2012-05-01 00:00:00,6,11.0,NaN,No,0
996,996,23144,1,Train,Rolla,MO,US,65401,PhD,Materials Science and Engineering,2012-05-01 00:00:00,5,7.0,No,No,0
997,997,23151,1,Train,Fort Wayne,IN,US,46825,Vocational,Legal Administration,1997-06-01 00:00:00,3,15.0,Yes,Yes,3
998,998,23156,1,Train,Warren,MI,US,48088,None,Numerous Business Management,NaN,2,34.0,No,No,0


### Build Dictionary that map userID to userIndex (both)

In [17]:
#Pre shape user ID to ID
UserID_to_Index = {}
Index_to_UserID = {}
for index in range(len(user)):
    UserID_to_Index[user.iloc[index][1]] = index
    Index_to_UserID[index] = user.iloc[index][1]

### Build M  

In [18]:
##### Adjacent list is used becuase matrix is sparsed avoid wasting space
train_job_adj_list={}   #jobID -> userID
train_user_adj_list={}  #userID -> jobID
for i in range (len(train_sample_app)):
    try:
#         if train_sample_app.iloc[i][1] == 22513:
#             print(train_sample_app.iloc[i][0])
#             print("find")
        train_job_adj_list[train_sample_app.iloc[i][1]].add(train_sample_app.iloc[i][0])
        train_user_adj_list[train_sample_app.iloc[i][0]].add(train_sample_app.iloc[i][1])
    except KeyError:
        train_job_adj_list[train_sample_app.iloc[i][1]] = {train_sample_app.iloc[i][0]}
        train_user_adj_list[train_sample_app.iloc[i][0]] = {train_sample_app.iloc[i][1]}
print(len(train_job_adj_list))
print(len(train_user_adj_list))
# print(train_job_adj_list)
# print(train_user_adj_list)

3189
754


In [19]:
test_job_adj_list={}   #jobID -> userID
test_user_adj_list={}  #userID -> jobID
for i in range (len(test_sample_app)):
    try:
        test_job_adj_list[test_sample_app.iloc[i][1]].add(test_sample_app.iloc[i][0])
        test_user_adj_list[test_sample_app.iloc[i][0]].add(test_sample_app.iloc[i][1])
    except KeyError:
        test_job_adj_list[test_sample_app.iloc[i][1]] = {test_sample_app.iloc[i][0]}
        test_user_adj_list[test_sample_app.iloc[i][0]] = {test_sample_app.iloc[i][1]}
print(len(test_job_adj_list))
print(len(test_user_adj_list))
# print(test_job_adj_list)
# print(test_user_adj_list)

837
374


In [20]:
# #### Build Normal M 
# binary_Matrix=np.zeros(shape=(total_jobs, len(user)))
# for index in range(len(sample_app)):
#     userID = sample_app.iloc[index][0]
#     jobID = sample_app.iloc[index][1]
#     i = jobID_to_index[jobID]
#     j = userID_to_Index[userID]
#     binary_Matrix[i][j] = 1

### Get Final Matrix M 
#### x=jobID, y=userID

In [21]:
Sim_Matrix=np.zeros(shape=(total_jobs, len(user)))

job_index = 0
for job_index in range(total_jobs):
    print(job_index)
    for user_index in range(len(user)):
        sigma_sum=0
        user_apply_job = False
        user_ID= Index_to_UserID[user_index]
        if(user_ID in train_user_adj_list):
            for job_ID in train_user_adj_list[user_ID]:
                if(JobID_to_Index[job_ID]==job_index):
                    user_apply_job = True
                    break
                sigma_sum = sigma_sum + sim(job_index, JobID_to_Index[job_ID]) * 1
        if(user_apply_job):
             Sim_Matrix[job_index][user_index]= 1
        else:
            Sim_Matrix[job_index][user_index]= sigma_sum
Sim_Matrix[20]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085


3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741


array([0.        , 0.01026658, 0.        , 0.        , 0.01547949,
       0.00665521, 0.0183917 , 0.        , 0.00676507, 0.0867809 ,
       0.04669433, 0.00268945, 0.00687303, 0.        , 0.02327269,
       0.00534292, 0.01570378, 0.        , 0.        , 0.        ,
       0.        , 0.0560457 , 0.01039991, 0.        , 0.01792824,
       0.06520248, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.0089954 , 0.        , 0.        ,
       0.        , 0.0016118 , 0.03676465, 0.        , 0.03534028,
       0.03424764, 0.01258531, 0.00364539, 0.        , 0.0277719 ,
       0.        , 0.        , 0.00871324, 0.        , 0.        ,
       0.00957322, 0.04072124, 0.00301296, 0.        , 0.00772035,
       0.        , 0.02143595, 0.00948711, 0.        , 0.02092437,
       0.04904138, 0.0051324 , 0.00502814, 0.        , 0.003544  ,
       0.        , 0.        , 0.00738385, 0.06821087, 0.03741063,
       0.01207479, 0.        , 0.        , 0.        , 0.01138

In [22]:
### Test Accuracy

In [26]:
test_job_adj_list
test_user_adj_list

count=0;
correct=0
for jobID in test_job_adj_list:
    job_index = JobID_to_Index[jobID]
    for userID in test_job_adj_list[jobID]:
        user_index = UserID_to_Index[userID]
        if Sim_Matrix[job_index][user_index] >= 0.1:
            correct = correct+1
        count = count + 1

print("total_jobs: ", total_jobs)
print("Test_sample_app: ", len(test_sample_app))
print("Accuracy: ", round(correct/count * 100,2), "%")

total_jobs:  3915
Test_sample_app:  857
Accuracy:  36.6 %
